In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout],[pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10]
pin = 0.5
pout = 0.01
seed = 0
alpha = 1e-3
lamda = 1e-3
eta = 1e-2
mu = 1e-2
no_users = sum(cluster_sizes)
batch_size = 20
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.         0.14285714 0.14285714 0.125      0.14285714
  0.14285714 0.14285714 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.125      0.14285714
  0.14285714 0.14285714 0.14285714 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.14285714]
 [0.14285714 0.         0.         0.         0.125      0.
  0.2        0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.2       ]
 [0.14285714 0.         0.         0.         0.125      0.
  0.         0.         0.16666667 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.125      0.125      0.125      0.125      0.         0.125
  0.         0.         0.125      0.         0.         0.
  0.         0.         0.         0.    

In [4]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [5]:
degree_list = [G.degree(i) for i in range(no_users)]
print(degree_list)

for i in G.neighbors(0):
    print(i)

[4, 6, 6, 4, 3, 7, 4, 3, 3, 5, 3, 2, 5, 4, 4, 5, 5, 1, 3, 5]
2
3
5
9


In [6]:
datapoints = {}
count = 0
W1 = np.array([2.0, 2.0])
W2 = np.array([-2.0, 2.0])
W = [W1, W2]
m = 200
n = 2
noise_sd = 0.001
for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n))
        label = np.dot(features, W[i]) + np.random.normal(0,noise_sd)
        datapoints[count] = {
                'features': features,
                'degree': node_degree(count, G),
                'label': label,
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i])
            }
        count += 1

In [7]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [8]:
class MLP_Net(nn.Module):
    def __init__(self, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(2, 1, bias=False)
        #self.fc2 = nn.Linear(4, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        #x = F.relu(self.fc1(x))
        output = self.fc1(x)
        #output = self.fc3(x)
        return output

In [9]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [10]:
model = MLP_Net(user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=50, shuffle=False)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for (x, y) in dataloader:
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)
        print(y.size())
        print(yhat.size())
        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss, grads_to_vector(model.parameters()), parameters_to_vector(model.parameters()))
        #optimizer.step()
        new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(10.3434, grad_fn=<MseLossBackward0>) tensor([ 5.7181, -3.4439]) tensor([0.5334, 0.2001], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(7.3859, grad_fn=<MseLossBackward0>) tensor([ 4.2482, -2.4084]) tensor([0.4763, 0.2345], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(11.5803, grad_fn=<MseLossBackward0>) tensor([ 6.0629, -4.8251]) tensor([0.4338, 0.2586], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
0 tensor(10.1169, grad_fn=<MseLossBackward0>) tensor([ 6.1634, -3.3119]) tensor([0.3732, 0.3068], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
1 tensor(8.6669, grad_fn=<MseLossBackward0>) tensor([ 5.2149, -3.1795]) tensor([0.3115, 0.3400], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
1 tensor(6.1873, grad_fn=<MseLossBackward0>) tensor([ 3.8691, -2.2310]) tensor([0.2594, 0.3717], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size(

12 tensor(1.2129, grad_fn=<MseLossBackward0>) tensor([ 2.0490, -1.2587]) tensor([-1.2085,  1.3612], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
13 tensor(1.0470, grad_fn=<MseLossBackward0>) tensor([ 1.7302, -1.2129]) tensor([-1.2290,  1.3737], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
13 tensor(0.7462, grad_fn=<MseLossBackward0>) tensor([ 1.2621, -0.8812]) tensor([-1.2463,  1.3859], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
13 tensor(1.1912, grad_fn=<MseLossBackward0>) tensor([ 1.8670, -1.6490]) tensor([-1.2589,  1.3947], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
13 tensor(1.0170, grad_fn=<MseLossBackward0>) tensor([ 1.8697, -1.1608]) tensor([-1.2776,  1.4112], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
14 tensor(0.8785, grad_fn=<MseLossBackward0>) tensor([ 1.5785, -1.1189]) tensor([-1.2963,  1.4228], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
14 tensor(0.6261

28 tensor(0.0879, grad_fn=<MseLossBackward0>) tensor([ 0.4829, -0.4754]) tensor([-1.8110,  1.8228], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
28 tensor(0.0732, grad_fn=<MseLossBackward0>) tensor([ 0.4746, -0.3424]) tensor([-1.8158,  1.8275], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
29 tensor(0.0639, grad_fn=<MseLossBackward0>) tensor([ 0.4000, -0.3312]) tensor([-1.8206,  1.8309], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
29 tensor(0.0456, grad_fn=<MseLossBackward0>) tensor([ 0.2845, -0.2493]) tensor([-1.8246,  1.8343], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
29 tensor(0.0739, grad_fn=<MseLossBackward0>) tensor([ 0.4414, -0.4375]) tensor([-1.8274,  1.8367], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
29 tensor(0.0615, grad_fn=<MseLossBackward0>) tensor([ 0.4332, -0.3155]) tensor([-1.8318,  1.8411], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
30 tensor(0.0537

41 tensor(0.0093, grad_fn=<MseLossBackward0>) tensor([ 0.1507, -0.1612]) tensor([-1.9419,  1.9393], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
41 tensor(0.0076, grad_fn=<MseLossBackward0>) tensor([ 0.1452, -0.1179]) tensor([-1.9434,  1.9409], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
42 tensor(0.0067, grad_fn=<MseLossBackward0>) tensor([ 0.1226, -0.1143]) tensor([-1.9449,  1.9421], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
42 tensor(0.0048, grad_fn=<MseLossBackward0>) tensor([ 0.0849, -0.0882]) tensor([-1.9461,  1.9432], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
42 tensor(0.0078, grad_fn=<MseLossBackward0>) tensor([ 0.1378, -0.1483]) tensor([-1.9469,  1.9441], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
42 tensor(0.0064, grad_fn=<MseLossBackward0>) tensor([ 0.1325, -0.1086]) tensor([-1.9483,  1.9456], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
43 tensor(0.0056

55 tensor(0.0005, grad_fn=<MseLossBackward0>) tensor([ 0.0253, -0.0310]) tensor([-1.9834,  1.9806], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
55 tensor(0.0008, grad_fn=<MseLossBackward0>) tensor([ 0.0435, -0.0501]) tensor([-1.9837,  1.9809], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
55 tensor(0.0007, grad_fn=<MseLossBackward0>) tensor([ 0.0405, -0.0371]) tensor([-1.9841,  1.9815], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
56 tensor(0.0006, grad_fn=<MseLossBackward0>) tensor([ 0.0347, -0.0360]) tensor([-1.9845,  1.9818], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
56 tensor(0.0004, grad_fn=<MseLossBackward0>) tensor([ 0.0230, -0.0286]) tensor([-1.9848,  1.9822], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
56 tensor(0.0007, grad_fn=<MseLossBackward0>) tensor([ 0.0398, -0.0461]) tensor([-1.9851,  1.9825], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
56 tensor(0.0006

70 tensor(6.6671e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0118, -0.0143]) tensor([-1.9958,  1.9945], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
70 tensor(4.9309e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0101, -0.0107]) tensor([-1.9960,  1.9947], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
71 tensor(4.6844e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0092, -0.0103]) tensor([-1.9961,  1.9948], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
71 tensor(3.3938e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0055, -0.0086]) tensor([-1.9962,  1.9949], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
71 tensor(5.6481e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0109, -0.0132]) tensor([-1.9962,  1.9950], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
71 tensor(4.1490e-05, grad_fn=<MseLossBackward0>) tensor([ 0.0092, -0.0098]) tensor([-1.9963,  1.9951], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([

84 tensor(5.9487e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0031, -0.0034]) tensor([-1.9988,  1.9983], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
84 tensor(4.5929e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0014, -0.0032]) tensor([-1.9989,  1.9983], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
84 tensor(7.3248e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0038, -0.0044]) tensor([-1.9989,  1.9983], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
84 tensor(4.8546e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0025, -0.0033]) tensor([-1.9989,  1.9984], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
85 tensor(5.1652e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0029, -0.0031]) tensor([-1.9990,  1.9984], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
85 tensor(4.0316e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0012, -0.0029]) tensor([-1.9990,  1.9984], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([

97 tensor(1.2626e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0005, -0.0010]) tensor([-1.9997,  1.9995], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
98 tensor(1.3893e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0012, -0.0009]) tensor([-1.9998,  1.9995], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
98 tensor(1.3341e-06, grad_fn=<MseLossBackward0>) tensor([ 8.6419e-05, -1.1982e-03]) tensor([-1.9998,  1.9995], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
98 tensor(1.5989e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0015, -0.0014]) tensor([-1.9998,  1.9995], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
98 tensor(1.2046e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0004, -0.0009]) tensor([-1.9998,  1.9995], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torch.Size([50, 1])
99 tensor(1.3158e-06, grad_fn=<MseLossBackward0>) tensor([ 0.0011, -0.0008]) tensor([-1.9998,  1.9995], grad_fn=<CatBackward0>)
torch.Size([50, 1])
torc

In [11]:
parameters_to_vector(model.parameters())

tensor([-1.9998,  1.9996], grad_fn=<CatBackward0>)

In [12]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [13]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.5)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                #grads = grads_to_vector(model.parameters())
                optimizer.step()
                train_loss += loss.item()*data.size(0)
                
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [14]:
# Preparing projection matrices
models = [MLP_Net(user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, first_run=True):
    #projected_weights = []
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        row, column = parameters_to_vector(models[j].parameters()).size()[0], parameters_to_vector(models[i].parameters()).size()[0]
                        mat = torch.zeros((row, column))
                        mat.fill_diagonal_(1.0 + 1.0 * float(np.random.randn(1)))
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[j].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[j][i], parameters_to_vector(models[j].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights)



In [15]:
print(projection_list[0])

[0, 0, tensor([[0.6216, 0.0000],
        [0.0000, 0.6216]]), tensor([[0.1495, 0.0000],
        [0.0000, 0.1495]]), 0, tensor([[2.0807, 0.0000],
        [0.0000, 2.0807]]), 0, 0, 0, tensor([[1.0993, 0.0000],
        [0.0000, 1.0993]]), 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [16]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [17]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(datapoints[model.user_id]['exact_weights'])).detach()

In [18]:
model = MLP_Net(user_id=0)

from torch.nn.utils import parameters_to_vector, vector_to_parameters

with torch.no_grad():    
    params = parameters_to_vector(model.parameters())

    print(params)

params *= 2.

vector_to_parameters(parameters=model.parameters(), vec=params)

parameters_to_vector(model.parameters())





tensor([-0.0820, -0.1822])


tensor([-0.1640, -0.3643], grad_fn=<CatBackward0>)

In [19]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    
    #Share and mix the local weights
    
    
    for i in range(no_users):
        weights = parameters_to_vector(dummy_models[i].parameters())
        mat_vec_sum = torch.zeros_like(weights)
        for j in G.neighbors(i):
            mat_vec_sum = torch.add(mat_vec_sum, parameters_to_vector(dummy_models[j].parameters()))
        
        new_weights = weights - mu * eta * (degree_list[i] * weights - mat_vec_sum)
        # Update real models
        vector_to_parameters(parameters=models[i].parameters(), vec=new_weights)
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    
    user_rel_error = 0
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        user_rel_error += rel_error(models[i])
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    total_rel_error.append(user_rel_error / no_users)
    #g_accuracy = sum(local_test_acc) / len(local_test_acc)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f,   Relative Error %2.5f"% (test_loss[-1], total_rel_error[-1]))

  0%|          | 2/2000 [00:00<01:52, 17.74it/s]

Training_loss 7.60723,   Relative Error 0.94956
Training_loss 7.57464,   Relative Error 0.94750
Training_loss 7.51970,   Relative Error 0.94407


  0%|          | 4/2000 [00:00<01:54, 17.47it/s]

Training_loss 7.50085,   Relative Error 0.94289


  0%|          | 6/2000 [00:00<02:03, 16.16it/s]

...

Training_loss 0.00217,   Relative Error 0.01590
Training_loss 0.00216,   Relative Error 0.01587


100%|█████████▉| 1999/2000 [04:26<00:00,  8.18it/s]

Training_loss 0.00216,   Relative Error 0.01585
Training_loss 0.00215,   Relative Error 0.01581


100%|██████████| 2000/2000 [04:26<00:00,  7.51it/s]

Training_loss 0.00214,   Relative Error 0.01579


In [20]:
#plot.plot(test_loss)
parameters_to_vector(models[19].parameters())

tensor([-1.9691,  1.9677], grad_fn=<CatBackward0>)

In [21]:
for j in G.neighbors(0):
    print(j)

2
3
5
9


In [22]:
parameters_to_vector(models[0].parameters())

tensor([1.9427, 1.9763], grad_fn=<CatBackward0>)

In [23]:
projection_list[0]

[0,
 0,
 tensor([[0.6216, 0.0000],
         [0.0000, 0.6216]]),
 tensor([[0.1495, 0.0000],
         [0.0000, 0.1495]]),
 0,
 tensor([[2.0807, 0.0000],
         [0.0000, 2.0807]]),
 0,
 0,
 0,
 tensor([[1.0993, 0.0000],
         [0.0000, 1.0993]]),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [24]:
projected_weights[0]

[0,
 0,
 tensor([0.3967, 0.2705]),
 tensor([-0.0940, -0.0868]),
 0,
 tensor([-1.2855, -0.8955]),
 0,
 0,
 0,
 tensor([-0.1048,  0.4911]),
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [25]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

In [26]:
np.save( 'training_loss_dfedu', test_loss)
np.save('relative_error_dfedu', total_rel_error)

In [ ]:
plt.plot(total_rel_error)